In [1]:
!pip install mlflow boto3 awscli optuna imbalanced-learn lightgbm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/26.4 MB 75.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 74.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 109.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 570.5/570.5 kB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.

In [ ]:
import mlflow
#Setup MLflow tracking server
mlflow.set_tracking_uri("http://18.117.193.162:5000/")

#set experiment
mlflow.set_experiment("Exp 5- Machine Learning Algorithms with HP tuning")

In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
from lightgbm import LGBMClassifier
import mlflow
import mlflow.sklearn
import optuna

In [ ]:
df = pd.read_csv('reddit_preprocessing.csv')

In [ ]:
#Step1: Remap the class labels from [-1, 0, 1] to [2, 0, 1]
df['category'] = df['category'].map({-1:2, 0:0, 1:1})

#Step2: Remove rows where the target labels(category) are NAN
df = df.dropna(subset=['category'])

In [7]:
ngram_range = (1,3)
max_features = 1000
vectorizer = TfidfVectorizer(ngram_range = ngram_range, max_features=max_features)

# Remove rows where 'clean_comment' is null
df = df.dropna(subset=['clean_comment'])

X=vectorizer.fit_transform(df['clean_comment'])
y = df['category']

smote = SMOTE(random_state = 42)
X_resampled, y_resampled = smote.fit_resample(X, y)

In [ ]:
#Step3: Train-test split before vectorization and resampling
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

In [9]:
#Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test, params = None):
  with mlflow.start_run():
     # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        #Log Hyperparamter
        if params:
          for key,value in params.items():
            mlflow.log_param(key, value)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")
        return accuracy

In [10]:
#Step6: Optuna Objective function for LightGBM
def objective_lightgbm(trial):
  #Hyperparameter space to explore
  n_estimators = trial.suggest_int('n_estimators', 100, 1000)
  learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
  max_depth = trial.suggest_int('max_depth',3,15)
  num_leaves = trial.suggest_int('num_leaves', 20, 150)
  min_child_samples = trial.suggest_int('min_child_samples', 10, 100)
  colsample_bytree = trial.suggest_float('colsample_bytree', 0.5, 1.0)
  subsample = trial.suggest_float('subsample', 0.5, 1.0)
  reg_alpha = trial.suggest_float('reg_alpha', 1e-4, 10.0, log=True)
  reg_lambda = trial.suggest_float('reg_lambda', 1e-4, 10.0, log=True)

  #Log trial parameters
  params = {
      'n_estimators': n_estimators,
      'learning_rate': learning_rate,
      'max_depth': max_depth,
      'num_leaves': num_leaves,
      'min_child_samples': min_child_samples,
      'colsample_bytree':colsample_bytree,
      'subsample': subsample,
      'reg_alpha': reg_alpha,
      'reg_lambda': reg_lambda
  }

  #Create LightGBM Model
  model = LGBMClassifier(
      learning_rate=learning_rate,
      n_estimators=n_estimators,
      max_depth=max_depth,
      num_leaves=num_leaves,
      min_child_samples=min_child_samples,
      colsample_bytree=colsample_bytree,
      subsample=subsample,
      reg_alpha=reg_alpha,
      reg_lambda=reg_lambda,
      random_state=42
  )

  accuracy = log_mlflow('LightGBM', model, X_train, X_test, y_train, y_test, params)
  return accuracy

In [11]:
#Step : Run Optuna for LightGBM, log the best model, and plot the importance of each other
def run_optuna_experiment():
  study = optuna.create_study(direction='maximize')
  study.optimize(objective_lightgbm, n_trials=50)

  #Get the best parameters
  best_params = study.best_params
  best_model = LGBMClassifier(
      n_estimators=best_params['n_estimators'],
      learning_rate=best_params['learning_rate'],
      max_depth=best_params['max_depth'],
      num_leaves=best_params['num_leaves'],
      min_child_samples=best_params['min_child_samples'],
      colsample_bytree=best_params['colsample_bytree'],
      subsample=best_params['subsample'],
      reg_alpha=best_params['reg_alpha'],
      reg_lambda=best_params['reg_lambda'],
      random_state=42
  )

  #log the best model with MLflow and print the classifiation report
  log_mlflow("LightGBM", best_model, X_train, X_test, y_train, y_test, params=best_params)

  #Plot parameter importance
  optuna.visualization.plot_param_importances(study).show()

  #Plot optimization history
  optuna.visualization.plot_optimization_history(study).show()

In [ ]:
#Run the experiment for LightBGM
run_optuna_experiment()

[I 2025-08-22 03:22:31,885] A new study created in memory with name: no-name-f7c69869-af77-449e-9f6a-b7c6698b9a5c


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.238797 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 98439
[LightGBM] [Info] Number of data points in the train set: 37848, number of used features: 945
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further s

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
2025/08/22 03:24:34 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/22 03:24:50 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run LightGBM_SMOTE_TFIDF_Trigrams at: http://18.117.193.162:5000/#/experiments/425591744193892905/runs/de26914905894ef0975b47b305fef3ba
🧪 View experiment at: http://18.117.193.162:5000/#/experiments/425591744193892905


[I 2025-08-22 03:24:55,975] Trial 0 finished with value: 0.7043965335024308 and parameters: {'n_estimators': 906, 'learning_rate': 0.00020378633310673392, 'max_depth': 14, 'num_leaves': 74, 'min_child_samples': 95, 'colsample_bytree': 0.6577812572155832, 'subsample': 0.679989587117588, 'reg_alpha': 0.0002746177456563449, 'reg_lambda': 0.22137872109017215}. Best is trial 0 with value: 0.7043965335024308.


...

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.257948 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 99046
[LightGBM] [Info] Number of data points in the train set: 37848, number of used features: 974
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further s

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
2025/08/22 04:17:16 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/22 04:17:29 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run LightGBM_SMOTE_TFIDF_Trigrams at: http://18.117.193.162:5000/#/experiments/425591744193892905/runs/81367caadd154347ada10c84a9dc8908
🧪 View experiment at: http://18.117.193.162:5000/#/experiments/425591744193892905


[I 2025-08-22 04:17:38,681] Trial 30 finished with value: 0.8136757556541957 and parameters: {'n_estimators': 989, 'learning_rate': 0.022669366040394448, 'max_depth': 11, 'num_leaves': 126, 'min_child_samples': 19, 'colsample_bytree': 0.6705074095876647, 'subsample': 0.7168612648248753, 'reg_alpha': 0.013317464312240512, 'reg_lambda': 0.6925982135946932}. Best is trial 18 with value: 0.8195941661382372.


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.250830 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 99060
[LightGBM] [Info] Number of data points in the train set: 37848, number of used features: 976
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further s